In [113]:
import os
import pandas as pd

## 获取数据集位置

In [114]:
floder_path=r"D:\trunz\Documents\DataScience\R_visualization\Data-Visualization_Teamwork\Data Set\北京空气质量"

filesList=[]
filesNum=[]
for root, dirs, files in os.walk(floder_path):
    filesNum.append(len(files))
    for file in files:
        if file.endswith(".csv"):
            file_path=os.path.join(root, file)
            filesList.append(file_path)

### 得到数据集路径与不同年份的index

In [115]:
filesNum=filesNum[1:-1]
filesNum

[627, 728, 732, 730, 730, 728, 730, 599, 511, 730]

In [116]:
fileindex=[0]
for num in filesNum:
    fileindex.append(fileindex[-1]+num)
fileindex

[0, 627, 1355, 2087, 2817, 3547, 4275, 5005, 5604, 6115, 6845]

## Load数据集并整合不同表格，得到逐年份的分时数据

In [117]:
DataList=[]
for i in range(len(filesNum)):
    df_all=pd.DataFrame()
    for file in filesList[fileindex[i]:fileindex[i+1]]:
        if os.path.getsize(file)==0:
            continue 
        df_temp=pd.read_csv(file)
        try:
            df_temp=df_temp[df_temp['type'].isin(['PM2.5','PM10','AQI','SO2','NO2','O3','CO'])]
        except Exception as e:
            continue
        df_all=pd.concat([df_all,df_temp])
        df_all=df_all.sort_values(by=['date','hour'])
    DataList.append(df_all)

In [119]:
# 保存 DataList 中的每个 DataFrame 为单独的 CSV 文件
output_dir1=os.path.join(os.path.pardir,'output_data/first clean')
os.makedirs(output_dir1,exist_ok=True)

# 保存 DataList 中的每个 DataFrame 为单独的 CSV 文件
for i, df in enumerate(DataList):
    output_file=os.path.join(output_dir1,f'Beijing_AirQuality_{i+2014}_hours.csv')
    df.to_csv(output_file,index=False,encoding='utf-8-sig')
    print(f"已保存文件: {output_file}")

已保存文件: ..\output_data/first clean\Beijing_AirQuality_2014_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2015_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2016_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2017_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2018_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2019_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2020_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2021_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2022_hours.csv
已保存文件: ..\output_data/first clean\Beijing_AirQuality_2023_hours.csv


In [120]:
# Dictionary mapping locations to districts
location_to_district = {
    '东城东四': '东城区', '东城天坛': '东城区', '西城官园': '西城区', '西城万寿西宫': '西城区', '朝阳奥体中心': '朝阳区',
    '朝阳农展馆': '朝阳区', '海淀万柳': '海淀区', '海淀四季青': '海淀区', '丰台小屯': '丰台区', '丰台云岗': '丰台区',
    '石景山古城': '石景山区', '石景山老山': '石景山区', '昌平镇': '昌平区', '昌平南邵': '昌平区', '定陵(对照点)': '昌平区',
    '延庆夏都': '延庆区', '延庆石河营': '延庆区', '怀柔镇': '怀柔区', '怀柔新城': '怀柔区', '密云镇': '密云区',
    '密云新城': '密云区', '平谷镇': '平谷区', '平谷新城': '平谷区', '顺义新城': '顺义区', '顺义北小营': '顺义区',
    '通州永顺': '通州区', '通州东关': '通州区', '大兴黄村': '大兴区', '大兴旧宫': '大兴区', '亦庄开发区': '大兴区',
    '京东南区域点': '大兴区', '门头沟双峪': '门头沟区', '门头沟三家店': '门头沟区', '房山良乡': '房山区', '房山燕山': '房山区','东四': '东城区', '天坛': '东城区', '官园': '西城区', '万寿西宫': '西城区', '奥体中心': '朝阳区', '农展馆': '朝阳区', '万柳': '海淀区',
    '北部新区': '海淀区', '植物园': '海淀区', '丰台花园': '丰台区', '云岗': '丰台区', '古城': '石景山区', '房山': '房山区', '大兴': '大兴区',
    '亦庄': '大兴区', '通州': '通州区', '顺义': '顺义区', '昌平': '昌平区', '门头沟': '门头沟区', '平谷': '平谷区', '怀柔': '怀柔区',
    '密云': '密云区', '延庆': '延庆区', '定陵': '昌平区', '八达岭': '延庆区', '密云水库': '密云区', '东高村': '顺义区', '永乐店': '通州区',
    '榆垡': '大兴区', '琉璃河': '房山区', '前门': '东城区', '永定门内': '东城区', '西直门北': '西城区', '南三环': '丰台区', '东四环': '朝阳区'
}
NewDataList=[]
# Function to merge columns by district
def merge_columns_by_district(df, location_to_district):
    district_data = {}
    for location, district in location_to_district.items():
        if location in df.columns:
            if district not in district_data:
                district_data[district] = []
            district_data[district].append(df[location])
    
    for district, columns in district_data.items():
        df[district] = pd.concat(columns, axis=1).mean(axis=1)
    
    column_to_drop = [location for location in location_to_district.keys() if location in df.columns]
    df = df.drop(columns=column_to_drop)
    return df

for data in DataList:
    NewDataList.append(merge_columns_by_district(data, location_to_district))

In [121]:
NewDataList[1]

,date,hour,type,东城区,西城区,朝阳区,海淀区,丰台区,石景山区,房山区,大兴区,通州区,顺义区,昌平区,门头沟区,平谷区,怀柔区,密云区,延庆区
0,20150101,0,PM2.5,8.375,7.700000,15.566667,5.700000,5.300000,7.9,73.25,13.366667,38.70,11.85,12.70,3.0,16.7,3.4,4.95,10.15
2,20150101,0,PM10,16.350,18.900000,44.233333,12.533333,14.650000,10.6,8.50,27.033333,57.05,28.55,33.80,10.2,33.4,13.9,21.70,30.40
4,20150101,0,AQI,49.000,43.000000,52.333333,39.666667,47.666667,28.0,56.00,57.000000,80.00,45.50,54.00,43.0,54.0,20.0,28.00,46.00
0,20150101,0,SO2,13.950,12.466667,15.533333,9.533333,5.500000,3.2,26.85,11.933333,31.25,7.25,6.40,2.7,9.1,2.0,9.55,15.70
2,20150101,0,NO2,28.050,19.100000,41.666667,7.766667,27.666667,4.0,34.35,20.066667,70.40,11.30,22.90,2.1,13.6,2.0,8.80,13.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,20151231,23,AQI,91.750,89.000000,99.333333,102.666667,96.666667,102.0,185.50,150.666667,147.50,88.00,72.00,83.0,129.0,65.0,68.00,80.00
184,20151231,23,SO2,26.250,44.333333,31.666667,39.333333,31.333333,34.0,32.50,44.333333,38.00,25.50,18.50,17.0,21.0,6.0,14.00,18.50
186,20151231,23,NO2,114.000,114.000000,118.000000,100.000000,101.000000,89.0,99.00,125.000000,102.50,84.00,88.00,81.0,61.0,44.0,49.50,80.00
188,20151231,23,O3,2.500,9.333333,15.000000,5.666667,7.000000,2.0,3.00,11.666667,9.50,7.00,2.50,2.0,19.0,2.0,12.00,7.50


## 异常值缺失值处理 & 获得不同污染物的逐日和逐月逐年数据

In [ ]:
for i,data in enumerate(NewDataList):
    for gas in ['AQI','CO','NO2','O3','PM10','PM2.5','SO2']:
        df=data[data['type']==gas]
    
        # 处理异常值和缺失值
        for station in df.columns[3:]:
            anomalous_index=df[df[station]>=1200].index
            df=df.drop(anomalous_index)
    
            # 检测是否存在缺失值
            missing_count=df[station].isnull().sum()
            if missing_count>0:
                # 使用插值法填充缺失值 
                df[station] = df[station].interpolate(method='linear')
                # 对仍有缺失值的部分，使用前向填充和后向填充补齐
                df[station] = df[station].fillna(method='ffill').fillna(method='bfill')
            
    
        # 逐小时数据导出
        output_dir2=os.path.join(os.path.pardir,'output_data/by_district/'+str(gas))
        os.makedirs(output_dir2,exist_ok=True)
        output_file=os.path.join(output_dir2,f'Beijing_AirQuality_{i+2014}{gas}_hours.csv')
        df.to_csv(output_file,index=False,encoding='utf-8-sig')
        
        # 逐日数据导出
        numeric_columns = df.columns[3:]
        df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
        daily_avg = df.groupby('date')[numeric_columns].mean().reset_index()  
        
        output_file=os.path.join(output_dir2,f'Beijing_AirQuality_{i+2014}{gas}_days.csv')
        daily_avg.to_csv(output_file,index=False,encoding='utf-8-sig')
        print(f"已保存文件: {output_file}")
        
        # 逐月数据导出
        df['month'] = df['date'].astype(str).str[:6]
        monthly_avg = df.groupby('month')[numeric_columns].mean().reset_index()
        
        output_file=os.path.join(output_dir2,f'Beijing_AirQuality_{i+2014}{gas}_months.csv')
        monthly_avg.to_csv(output_file,index=False,encoding='utf-8-sig')
        print(f"已保存文件: {output_file}")
    

In [123]:
# 逐年数据
for gas in ['AQI','CO','NO2','O3','PM10','PM2.5','SO2']:
    output_dir3=os.path.join(os.path.pardir,'output_data/by_district/'+str(gas))
    yearly_data=pd.DataFrame()
    monthly_data=pd.DataFrame()
    for year in range(2014,2024):
        file_path=os.path.join(output_dir3,f'Beijing_AirQuality_{year}{gas}_months.csv')
        df=pd.read_csv(file_path)
        monthly_data=pd.concat([monthly_data,df],ignore_index=True,join='outer')
    
    numeric_columns = monthly_data.columns[3:]
    monthly_data['year']=monthly_data['month'].astype(str).str[:4]
    yearly_avg=monthly_data.groupby('year')[numeric_columns].mean().reset_index()
    
    output_file=os.path.join(output_dir3,f'Beijing_AirQuality_{gas}_years.csv')
    yearly_avg.to_csv(output_file,index=False,encoding='utf-8-sig')
    print(f"已保存文件: {output_file}")

C:\Users\trunz\AppData\Local\Temp\ipykernel_5960\182866041.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  monthly_data=pd.concat([monthly_data,df],ignore_index=True,join='outer')


已保存文件: ..\output_data/by_district/AQI\Beijing_AirQuality_AQI_years.csv
已保存文件: ..\output_data/by_district/CO\Beijing_AirQuality_CO_years.csv
已保存文件: ..\output_data/by_district/NO2\Beijing_AirQuality_NO2_years.csv
已保存文件: ..\output_data/by_district/O3\Beijing_AirQuality_O3_years.csv
已保存文件: ..\output_data/by_district/PM10\Beijing_AirQuality_PM10_years.csv
已保存文件: ..\output_data/by_district/PM2.5\Beijing_AirQuality_PM2.5_years.csv
已保存文件: ..\output_data/by_district/SO2\Beijing_AirQuality_SO2_years.csv


In [124]:
# 逐年逐月数据
for gas in ['AQI','CO','NO2','O3','PM10','PM2.5','SO2']:
    output_dir4=os.path.join(os.path.pardir,'output_data/by_district/'+str(gas))
    allmonths_data=pd.DataFrame()
    for year in range(2014,2024):
        file_path=os.path.join(output_dir4,f'Beijing_AirQuality_{year}{gas}_months.csv')
        df=pd.read_csv(file_path)
        allmonths_data=pd.concat([allmonths_data,df],ignore_index=True,join='outer')
    
    output_file=os.path.join(output_dir4,f'Beijing_AirQuality_{gas}_allmonths.csv')
    allmonths_data.to_csv(output_file,index=False,encoding='utf-8-sig')
    print(f"已保存文件: {output_file}")

已保存文件: ..\output_data/by_district/AQI\Beijing_AirQuality_AQI_allmonths.csv
已保存文件: ..\output_data/by_district/CO\Beijing_AirQuality_CO_allmonths.csv
已保存文件: ..\output_data/by_district/NO2\Beijing_AirQuality_NO2_allmonths.csv
已保存文件: ..\output_data/by_district/O3\Beijing_AirQuality_O3_allmonths.csv
已保存文件: ..\output_data/by_district/PM10\Beijing_AirQuality_PM10_allmonths.csv
已保存文件: ..\output_data/by_district/PM2.5\Beijing_AirQuality_PM2.5_allmonths.csv
已保存文件: ..\output_data/by_district/SO2\Beijing_AirQuality_SO2_allmonths.csv


C:\Users\trunz\AppData\Local\Temp\ipykernel_5960\3679534774.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  allmonths_data=pd.concat([allmonths_data,df],ignore_index=True,join='outer')


In [126]:
# 删除冗杂的逐月数据
for gas in ['AQI','CO','NO2','O3','PM10','PM2.5','SO2']:
    for year in range(2014,2024):
        output_dir4=os.path.join(os.path.pardir,'output_data/by_district/'+str(gas))
        file_path=os.path.join(output_dir4,f'Beijing_AirQuality_{year}{gas}_months.csv')
        os.remove(file_path)
        print(f"已删除文件: {file_path}")

已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2014AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2015AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2016AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2017AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2018AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2019AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2020AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2021AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2022AQI_months.csv
已删除文件: ..\output_data/by_district/AQI\Beijing_AirQuality_2023AQI_months.csv
已删除文件: ..\output_data/by_district/CO\Beijing_AirQuality_2014CO_months.csv
已删除文件: ..\output_data/by_district/CO\Beijing_AirQuality_2015CO_months.csv
已删除文件: ..\output_data/by_district/CO\Beijing_AirQuality_2016CO_months.csv
已删除文件: ..\output_d

In [128]:
# Release all variables
del DataList, NewDataList, df, df_all, df_temp, daily_avg, monthly_avg, yearly_avg, allmonths_data

NameError: name 'DataList' is not defined